In [68]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [123]:
import psycopg2 
import pandas as pd

In [124]:
def create_database():
    #connect to default database
    conn = psycopg2.connect('host=localhost dbname=postgres user=postgres password=Mradal')
    conn.set_session(autocommit = True)
    curr = conn.cursor()

    # create a database with UTF8 encoding
    # Create a database with UTF8 encoding
    try:
        curr.execute('DROP DATABASE IF EXISTS EcommerceSales')
    except psycopg2.Error as e:
        print('Error dropping database:', e)

    print('Creating a new database...')
    try:
        curr.execute('CREATE DATABASE EcommerceSales WITH ENCODING "UTF8"')
        print('Database creation done!')
    except psycopg2.Error as e:
        print('Error creating database:', e)

    # Close the connection to the default database
    conn.close()

    # Connect to the newly created SalesDataStore database
    conn = psycopg2.connect(
        host='localhost',
        dbname='ecommercesales',
        user='postgres',
        password='Mradal'
    )
    curr = conn.cursor()

    return curr, conn

In [125]:
def drop_tables(cur, conn):
    for query in drop_table_queries:
        cur.execute(query)
        conn.commit()

In [126]:
def create_tables(cur, conn):
    for query in create_table_queries:
        cur.execute(query)
        conn.commit()

In [127]:
CustomerData = pd.read_csv('customers.csv')

In [128]:
CustomerData.head()

,customer_id,first_name,last_name,email,phone_number,address,product_id
0,1,Michelle,Williams,bdelgado@example.org,530-792-2293,"1161 Jacqueline Rue Apt. 321\nMcdonaldfort, AS...",853
1,2,Mackenzie,Finley,isabelrodriguez@example.com,261.752.9989x38750,"7723 Bobby Gardens\nBarnettmouth, MN 68686",783
2,3,Timothy,Graves,cheryl72@example.net,429-968-2889x40897,"2364 Abigail Estates\nLake Danielmouth, NY 28371",445
3,4,Michael,Hall,davidparker@example.com,311-423-7861,"9688 Freeman Ramp\nSouth Jared, NM 17447",563
4,5,Jacqueline,Rogers,rodriguezallen@example.net,(784)466-4383,"503 Mackenzie Parkways\nEast Patrick, MS 83607",70


In [129]:
ProductData = pd.read_csv('products.csv')

In [130]:
ProductData.head()

,product_id,product_name,description,price,category
0,1,Support,Camera TV provide picture staff gun man.,48.08,create
1,2,Not,National argue civil.,93.81,change
2,3,Such,Social grow pretty common couple of save.,140.79,design
3,4,Plant,Ready visit prove recognize rule our.,87.96,four
4,5,Step,Machine control significant while policy bed.,55.57,scene


In [131]:
SalesData = pd.read_csv('sales.csv')
SalesData.head()

,sale_id,customer_id,product_id,sale_date,quantity,total_amount
0,1,177,141,2023-08-09 13:33:18,3,283.668881
1,2,772,577,2023-07-22 18:33:03,1,195.354914
2,3,531,920,2023-05-02 20:49:55,4,307.912755
3,4,941,10,2022-11-28 05:23:54,4,693.312641
4,5,508,981,2023-08-15 20:07:39,4,503.069913


In [132]:
# Create the database
cur, conn = create_database()

Creating a new database...
Database creation done!


In [133]:
#Create Product table
product_table_create = ("""CREATE TABLE IF NOT EXISTS products (
    product_id INT PRIMARY KEY,
    product_name VARCHAR(255),
    description TEXT,
    price DECIMAL(10, 2),
    category VARCHAR(255)
);""")

cur.execute(product_table_create)
conn.commit()

In [134]:
#create Customers table
customer_table_create = ("""CREATE TABLE IF NOT EXISTS customers (
    customer_id INT PRIMARY KEY,
    first_name VARCHAR(255),
    last_name VARCHAR(255),
    email VARCHAR(255),
    phone_number VARCHAR(50),
    address TEXT,
    product_id INT REFERENCES products(product_id)
);""")

cur.execute(customer_table_create)
conn.commit()


In [135]:
#create sales table
sales_table_create = ("""CREATE TABLE IF NOT EXISTS sales (
    sale_id SERIAL PRIMARY KEY,
    customer_id INT REFERENCES customers(customer_id),
    product_id INT REFERENCES products(product_id),
    sale_date TIMESTAMP,
    quantity INT,
    total_amount DECIMAL(10, 2)
);""")

cur.execute(sales_table_create)
conn.commit()


![Screenshot](ss1.jpg)


In [136]:
print(ProductData.dtypes)
ProductData.head()

product_id        int64
product_name     object
description      object
price           float64
category         object
dtype: object


,product_id,product_name,description,price,category
0,1,Support,Camera TV provide picture staff gun man.,48.08,create
1,2,Not,National argue civil.,93.81,change
2,3,Such,Social grow pretty common couple of save.,140.79,design
3,4,Plant,Ready visit prove recognize rule our.,87.96,four
4,5,Step,Machine control significant while policy bed.,55.57,scene


In [137]:
ProductData['product_id'] = ProductData['product_id'].astype(int)
ProductData['product_name'] = ProductData['product_name'].astype(str)
ProductData['description'] = ProductData['description'].astype(str)
ProductData['price'] = ProductData['price'].astype(float)
ProductData['category'] = ProductData['category'].astype(str)

product_table_insert = ("""
    INSERT INTO products (product_id, product_name, description, price, category)
    VALUES (%s, %s, %s, %s, %s)
""")


In [138]:
# Loop through the rows and insert data row by row
for i, row in ProductData.iterrows():
    try:
        # Print the data for debugging
        #print(f"Inserting row {i}: {list(row)}")

        # Execute the INSERT statement with the current row's data
        cur.execute(product_table_insert, list(row))
        conn.commit()
    except Exception as e:
        # Handle the error (e.g., log it) and continue to the next row
        print(f"Error inserting row {i}: {str(e)}")
        conn.rollback()  # Roll back the transaction to continue with the next row


In [120]:
customer_table_insert = ("""
    INSERT INTO customers (customer_id, first_name, last_name, email, phone_number, address, product_id)
    VALUES (%s, %s, %s, %s, %s, %s, %s)
""")



In [139]:
for i, row in CustomerData.iterrows():
    try:
        # Print the data for debugging
        #print(f"Inserting row {i}: {list(row)}")

        # Execute the INSERT statement with the current row's data
        cur.execute(customer_table_insert, list(row))
        conn.commit()
    except Exception as e:
        # Handle the error (e.g., log it) and continue to the next row
        print(f"Error inserting row {i}: {str(e)}")
        conn.rollback()  # Roll back the transaction to continue with the next row


In [143]:
sales_table_insert = ("""
    INSERT INTO sales (sale_id, customer_id, product_id, sale_date, quantity, total_amount)
    VALUES (%s, %s, %s, %s, %s, %s)
""")

In [144]:
for i, row in SalesData.iterrows():
    try:
        # Print the data for debugging
        #print(f"Inserting row {i}: {list(row)}")

        # Execute the INSERT statement with the current row's data
        cur.execute(sales_table_insert, list(row))
        conn.commit()
    except Exception as e:
        # Handle the error (e.g., log it) and continue to the next row
        print(f"Error inserting row {i}: {str(e)}")
        conn.rollback()  # Roll back the transaction to continue with the next row